# Filter Change MTPtg/Rotator Integration Test

This notebook performs a filter change scenario integration test between the Maint Telescope pointing component (MTPtg) and the Rotator with the Camera Cable Wrap tracking the Rotator. It includes enough boilerplate to allow the test to run at any time by getting current time information from the pointing and computing appropriate coordinates to slew.

In [ ]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

from astropy.utils import iers
iers.conf.auto_download = False

In [ ]:
test_message = "Filter Change MTPtg_Rotator_CCW Integration Test"

In [ ]:
d = salobj.Domain()

In [ ]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "Rotator")
mtptg = salobj.Remote(d, "MTPtg")

In [ ]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task)

In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

# Setup Pointing Target

In [ ]:
# Here is a small trick to get the ra before transit. Get `timeAndDate` from the pointing component, then use `RA = lst - delta`.

time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=5.)

lst = Angle(time_data.lst, unit=u.hour)
delta = Angle(10., unit=u.arcminute)
ra = lst + delta
print("=======================")
print(f"LST: {lst}")
print(f"delta: {delta}")
print(f"RA: {ra}")
print("=======================")

dec = location.lat - Angle(6., unit=u.deg)
print(f"Dec: {dec}")

target_name="Rotator test"
radec = ICRS(ra, dec)

curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print(180.-alt_az.alt.deg)

# Perform First Track

In [ ]:
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message}")

try:
    await rot.cmd_trackStart.start(timeout=30.)
except salobj.AckError:
    pass

ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.,
    rotFrame=ATPtg.RotFrame.TARGET,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print("Waiting 30s")

await asyncio.sleep(30.)

print("Stop track")

await mtptg.cmd_stopTracking.start(timeout=30.)

await asyncio.sleep(0.1)

await rot.cmd_stop.start(timeout=30.)

# Perform Filter Change - Move Command

# Perform Filter Change - Track Command

In [ ]:
await asyncio.sleep(0.1)

print("Track to 0.0 deg for filter change")

tel = rot.tel_Application.get()
cpos = tel.Position
print(cpos)

vel = 0.068
dt = (0 + cpos) / vel
dpos = vel * 0.1
steps = int(dt / 0.1)

try:
    await rot.cmd_trackStart.start(timeout=30.)
except salobj.AckError:
    pass

for i in range(steps):
    pos = cpos - i*dpos
    await rot.cmd_track.set_start(
        angle=pos,
        velocity=vel,
        tai=salobj.current_tai(),
        timeout=10.
    )
    await asyncio.sleep(0.1)
    
await rot.cmd_stop.start(timeout=30.)

print("Rotator in position and waiting 120s")

await asyncio.sleep(120.)

# Perform Second Track

In [ ]:
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=5.)

lst = Angle(time_data.lst, unit=u.hour)
delta = Angle(10., unit=u.arcminute)
ra = lst + delta
print("=======================")
print(f"LST: {lst}")
print(f"delta: {delta}")
print(f"RA: {ra}")
print("=======================")

dec = location.lat + Angle(6., unit=u.deg)
print(f"Dec: {dec}")

target_name="Rotator test"
radec = ICRS(ra, dec)

curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print(180.-alt_az.alt.deg)

print("Start second track")

try:
    await rot.cmd_trackStart.start(timeout=30.)
except salobj.AckError:
    pass

ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=0.,
    rotFrame=ATPtg.RotFrame.TARGET,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print("Waiting 30s")

await asyncio.sleep(30.)

await mtptg.cmd_stopTracking.start(timeout=30.)

await asyncio.sleep(0.1)

await rot.cmd_stop.start(timeout=30.)

# Move to Zero - Move Command

# Move to Zero - Track Command

In [ ]:
await asyncio.sleep(0.1)

print("Track to 0.0 deg for ending position")

tel = rot.tel_Application.get()
cpos = tel.Position
print(cpos)

vel = 0.068
dt = (0 - cpos) / vel
dpos = vel * 0.1
steps = int(dt / 0.1)

try:
    await rot.cmd_trackStart.start(timeout=30.)
except salobj.AckError:
    pass

for i in range(steps):
    pos = cpos + i*dpos
    await rot.cmd_track.set_start(
        angle=pos,
        velocity=vel,
        tai=salobj.current_tai(),
        timeout=10.
    )
    await asyncio.sleep(0.1)
    
await rot.cmd_stop.start(timeout=30.)

print("Test complete")

script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"END - {test_message}")